# 2_Upload_to_Pinecone Notebook

전처리된 데이터를 임베딩하고 Pinecone 인덱스에 업로드하는 노트북입니다.

## 1. 환경 준비
- Markdown 문서를 읽어 파싱하는 유틸을 불러옵니다.
- OpenAI/Pinecone 클라이언트를 초기화합니다.

In [4]:
# Markdown 헤더 기준으로 문서 분할 도구 설정
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 헤더 레벨 → 메타데이터 키 매핑
headers_to_split_on = [
    ("#", "title"),
    ("##", "chapter"),
    ("###", "section"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

## 2. Pinecone 임베딩 업로드
- 임베딩 모델을 사용해 문서를 벡터화하고 지정한 인덱스에 업서트합니다.

In [5]:
# Pinecone 벡터스토어와 임베딩 초기화
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
import os

load_dotenv()  # .env 로드

pinecone = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))  # Pinecone 클라이언트

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")  # 임베딩 모델
index_name = "inhouse-python-index"  # 기존에 생성된 인덱스 이름
index = pinecone.Index(index_name)  # 인덱스 핸들

vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings,
)  # LangChain VectorStore 래퍼

## 3. 업로드 검증
- 업로드된 문서를 샘플 조회하여 검색 품질을 확인합니다.

In [8]:
# 변환된 Markdown 파일들을 분할하여 업로드
import os

input_dir = './output'  # data_processing에서 생성된 결과 폴더

for filename in os.listdir(input_dir):
    if filename.endswith('.md'):
        md_path = os.path.join(input_dir, filename)
        # UTF-8로 마크다운 읽기
        with open(md_path, 'r', encoding='utf-8') as f:
            markdown_text = f.read()

        # 헤더 기반으로 문서를 섹션 단위로 분할
        docs = markdown_splitter.split_text(markdown_text)
        for doc in docs:
            # 원본 파일명을 메타데이터로 보존 (확장자 제거)
            doc.metadata['source'] = filename.replace('.md', '')

        # Pinecone 벡터스토어에 업로드
        vector_store.add_documents(docs)


## 4. 후속 작업
- 필요 시 추가 테스트나 운영 노하우를 기록합니다.

In [9]:
# 검색기 생성 (k-NN 질의용)
retriever = vector_store.as_retriever()

In [10]:
# 예시 질의: 경조 휴가 일수 확인
retriever.invoke("본인 결혼식 경조 휴가는 얼마인가요?")

[Document(id='fba192c3-86fb-45aa-bf94-168050580950', metadata={'chapter': '제 7 조 (경조 휴가)', 'source': 'employee_benefits_and_welfare_guide'}, page_content='다음의 경조사유에 대해 휴가를 부여한다:  \n| 경조 사유    | 휴가 일수 |\n|--------------|-----------|\n| 본인 결혼    | 5 일      |\n| 자녀 결혼    | 2 일      |\n| 부모 사망    | 5 일      |\n| 배우자 사망  | 7 일      |\n| 조부모 사망  | 3 일      |\n| 자녀 출생    | 3 일      |\n| 본인 생일    | 1 일      |\n| 결혼 기념일  | 1 일      |'),
 Document(id='02d35258-b778-44ae-99fb-5fe45ba86dc0', metadata={'chapter': '제 3장 근로 시간 및 휴가', 'section': '제 6조 (근로 시간)', 'source': 'employee_handbook_and_hr_policy', 'title': '직원 핸드북 및 인사 정책'}, page_content='- 기본 근무 시간: 월요일~금요일, 오전 9시 ~ 오후 6시\n- 점심 시간: 오후 12시 ~ 오후 1시\n- 유연 근무 가능 (사전 승인 필요)  \n제 7조 (지각, 결근)\n• 지각: 정해진 근무 시간 이후 도착할 경우\n• 결근: 사전에 신청 없음 출력하지 않는 경우\n• 3회 지각 시 1회 결근으로 처리  \n제 8조 (연차 유급 휴가)\n• 연차 유급 휴가는 근속 연수에 따라 차등 지급한다.\n• 연차 휴가 지급 기준:  \n| 근속 연수 | 연차 휴가 일수 |\n|-----------|----------------|\n| 1년 미만 | 1 일           |\n| 1~3년    | 15 일          |\n| 4~